In [0]:
from __future__ import division
import warnings
warnings.filterwarnings('ignore')

! pip install allennlp

import os
import sys
import time
import nltk
import math
import random
import pickle
import zipfile
import numpy as np
import pandas as pd
nltk.download('all')
import tensorflow as tf
from google.colab import drive
from collections import Counter
from keras.utils import Sequence
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from allennlp.commands.elmo import ElmoEmbedder
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from keras.optimizers import Adam, Adamax, Nadam, Adagrad
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

Using TensorFlow backend.


In [0]:
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

# Function Definitions

In [0]:
porter = PorterStemmer()
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [0]:
def clean(df,tasks,dtype,fname):
  column = 'query'
  #Lowercase conversion
  df[column] = df[column].apply(lambda x: " ".join(x.lower() for x in x.split()))
  print(column+": Converted to lowercase")
  
  #Tokenization
  df[column] = df[column].apply(word_tokenize)
  df[column] = df[column].apply(lambda x: " ".join(x))
  print(column+": Tokenized")
  if('- ' in tasks):
    #Split a-b into a and b
    df[column] = df[column].str.replace('-',' ')
    print(column+": - Replaced")
  
  elif('-_' in tasks):
    #Split a-b into a and b
    df[column] = df[column].str.replace('-','_')
    print(column+": - Replaced")
  
  if('punct' in tasks):
    #Removing punctuations
    df[column] = df[column].str.replace('[^\w\s]',' ')
    print(column+": Removed punctions ")
  
  if('num' in tasks):
    #Replacing numbers
    df[column] = df[column].str.replace('[0-9]','#')
    print(column+": Replaced Numbers ")
  
  #if('stop' in tasks):
    #Removing Stop Words
    #df[column] = df[column].apply(lambda x: " ".join(word for word in x if word not in stop_words))
    #print(column+": StopWords Removed")
  
  if('lemma' in tasks):
    #Lemmatization - root words
    df[column] = df[column].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split()]))
    print(column+": Root words Lemmatized")
  
  column = 'response'
  #Lowercase conversion
  df[column] = df[column].apply(lambda x: " ".join(k.lower() for k in x.split()))
  print(column+": Converted to lowercase")
  
  #Tokenization
  df[column] = df[column].apply(lambda x: " ".join(word_tokenize(x)))
  print(column+": Tokenized")
  
  if('- ' in tasks):
    #Split a-b into a and b
    df[column] = df[column].str.replace('-',' ')
    print(column+": - Replaced")
  
  elif('-_' in tasks):
    #Split a-b into a and b
    df[column] = df[column].str.replace('-','_')
    print(column+": - Replaced")
  
  if('punct' in tasks):
    #Removing punctuations
    df[column] = df[column].str.replace('[^\w\s]',' ')
    print(column+": Removed punctions ")
  
  if('num' in tasks):
    #Replacing numbers
    df[column] = df[column].str.replace('[0-9]','#')
    print(column+": Replaced Numbers ")
  
  #if('stop' in tasks):
    #Removing Stop Words
    #df[column] = df[column].apply(lambda x: " ".join(word for word in x if word not in stop_words))
    #print(column+": StopWords Removed")
  
  if('lemma' in tasks):
    #Lemmatization - root words
    df[column] = df[column].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split()]))
    print(column+": Root words Lemmatized")
    
  df['Row_no'] = df.index  
  print(df.head())
  df.to_csv("./gdrive/My Drive/Microsoft AI/Finalised Version/"+dtype+"/"+fname+".csv",sep=',',index=None)

  

In [0]:
archive = zipfile.ZipFile('gdrive/My Drive/Microsoft AI/Junk/Just For now/data.zip', 'r')
data = pd.read_csv(archive.open("data.tsv"),sep='\t',header=None)
data.columns = ['queryID','query','response','label','labelID']

In [0]:
data.head()

,queryID,query,response,label,labelID
0,131,. what is a corporation?,A company is incorporated in a specific nation...,0,0
1,131,. what is a corporation?,"Today, there is a growing community of more th...",0,1
2,131,. what is a corporation?,"Corporation definition, an association of indi...",0,2
3,131,. what is a corporation?,Examples of corporation in a Sentence. 1 He w...,0,3
4,131,. what is a corporation?,1: a government-owned corporation (as a utilit...,0,4


In [0]:
tasks = ['- ','punct','lemma','num']
clean(data,tasks,'Data_for_ELMO','clean_train_elmo')

response: Converted to lowercase
response: Tokenized
response: - Replaced
response: Removed punctions 
response: Replaced Numbers 
response: Root words Lemmatized
   queryID                     query  \
0      131  . what is a corporation?   
1      131  . what is a corporation?   
2      131  . what is a corporation?   
3      131  . what is a corporation?   
4      131  . what is a corporation?   

                                            response  label  labelID  Row_no  
0  a company is incorporated in a specific nation...      0        0       0  
1  today there is a growing community of more tha...      0        1       1  
2  corporation definition an association of indiv...      0        2       2  
3  example of corporation in a sentence # he work...      0        3       3  
4  # a government owned corporation a a utility o...      0        4       4  


In [0]:
data = pd.read_csv("./gdrive/My Drive/Microsoft AI/Finalised Version/Data_for_ELMO/clean_train_elmo.csv",sep=',')

In [0]:
data.head()

,queryID,query,response,label,labelID,Row_no
0,131,. what is a corporation?,a company is incorporated in a specific nation...,0,0,0
1,131,. what is a corporation?,today there is a growing community of more tha...,0,1,1
2,131,. what is a corporation?,corporation definition an association of indiv...,0,2,2
3,131,. what is a corporation?,example of corporation in a sentence # he work...,0,3,3
4,131,. what is a corporation?,# a government owned corporation a a utility o...,0,4,4


In [0]:
tasks = ['- ','punct','lemma','num']
clean(data,tasks,'Data_for_ELMO','clean_train_elmo')

query: Converted to lowercase
query: Tokenized
query: - Replaced
query: Removed punctions 
query: Replaced Numbers 
query: Root words Lemmatized
   queryID                  query  \
0      131  what is a corporation   
1      131  what is a corporation   
2      131  what is a corporation   
3      131  what is a corporation   
4      131  what is a corporation   

                                            response  label  labelID  Row_no  
0  a company is incorporated in a specific nation...      0        0       0  
1  today there is a growing community of more tha...      0        1       1  
2  corporation definition an association of indiv...      0        2       2  
3  example of corporation in a sentence # he work...      0        3       3  
4  # a government owned corporation a a utility o...      0        4       4  


In [0]:
tdata = pd.read_csv("gdrive/My Drive/Microsoft AI/eval2_unlabelled.tsv",sep='\t',header=None)
tdata.columns = ['queryID','query','response','labelID']

In [0]:
tasks = ['- ','punct','lemma','num']
clean(tdata,tasks,'Data_for_ELMO','clean_test2_elmo')

query: Converted to lowercase
query: Tokenized
query: - Replaced
query: Removed punctions 
query: Replaced Numbers 
query: Root words Lemmatized
response: Converted to lowercase
response: Tokenized
response: - Replaced
response: Removed punctions 
response: Replaced Numbers 
response: Root words Lemmatized
   queryID              query  \
0  1136966  ffffff color code   
1  1136966  ffffff color code   
2  1136966  ffffff color code   
3  1136966  ffffff color code   
4  1136966  ffffff color code   

                                            response  labelID  Row_no  
0  color hex is a easy to use tool to get the col...        0       0  
1  ffffff color conversion the hexadecimal color ...        1       1  
2  cs code color preview color scheme color picke...        2       2  
3  color hex color code color hex give informatio...        3       3  
4  color hex color code color hex give informatio...        4       4  


In [0]:
! wget "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
! wget "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

In [0]:
options_file = "./elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "./elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

In [0]:
elmo = ElmoEmbedder(options_file, weight_file, -1)

12/27/2018 16:56:27 - INFO - allennlp.commands.elmo -   Initializing ELMo.


In [0]:
data['query'][0]

'. what is a corporation?'

In [0]:
def get_elmo_embed(elmo,df,column,fname,dname):
  vecs = []
  for i in range(df.shape[0]):
    if(i%1000 == 0 and i!= 0):
      pickle.dump(vecs,open("./gdrive/My Drive/Microsoft AI/Finalised Version/"+dname+"/ELMO Embeddings/"+fname+"_"+str(i)+".pkl","wb"))
      del vecs
      vecs = []
    vecs.append(elmo.embed_sentence(word_tokenize(data['query'][i]))[2].shape)

(6, 1024)

In [0]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))


def get_basic_features(df,ltype,dtype):
  
  column='query'
  df[column[0]+'_word_count'] = df[column].apply(lambda x: len(str(x).split(" ")))
  print(column+"Word Count Done")
  df[column[0]+'_char_count'] = df[column].str.len()
  print(column+"Char Count Done")
  df[column[0]+'_avg_word'] = df[column].apply(lambda x: avg_word(x))
  print(column+"Avg Word Length Done")
  
  column='response'
  df[column[0]+'_word_count'] = df[column].apply(lambda x: len(str(x).split(" ")))
  print(column+"Word Count Done")
  df[column[0]+'_char_count'] = df[column].str.len()
  print(column+"Char Count Done")
  df[column[0]+'_avg_word'] = df[column].apply(lambda x: avg_word(x))
  print(column+"Avg Word Length Done")
  
  df.to_csv("./gdrive/My Drive/Microsoft AI/Finalised Version/"+dtype+\
            "/Latest Features/"+ltype+"_Basic_features.csv",sep=',',index=None)

In [0]:
def get_cosine_sim(df,ltype,dtype):
  cosine_sim = []
  for i in range(0,df.shape[0],10):
    if(i%10000 == 0):
      print(i)
    sent = (df['query'][i], *(df['response'][i:i+10].T))
    text = [t for t in sent]
    vectorizer = TfidfVectorizer()
    vect = vectorizer.fit_transform(text).toarray()
    vectors = [t for t in vect]
    cosine_sim.extend(cosine_similarity(vectors)[0,1:].ravel())
  cosine_sim = pd.Series(cosine_sim).rename("Cosine")
  cosine_sim.to_csv("./gdrive/My Drive/Microsoft AI/Finalised Version/"+dtype+\
                    "/Latest Features/"+ltype+"_Cosine_Similarity.csv",sep=',',index=None,header=None)

In [0]:
def get_jaccard_sim(df,ltype,dtype):
  jac_sim = []
  for i in range(df.shape[0]):
    if(i%10000 == 0):
      print(i)
    a = set(df['query'][i].split())
    b = set(df['response'][i].split())
    c = a.intersection(b)
    jac_sim.append(float(len(c)) / (len(a) + len(b) - len(c)))
  jac_sim = pd.Series(jac_sim).rename("Jaccard")
  print(jac_sim.shape)
  jac_sim.to_csv("./gdrive/My Drive/Microsoft AI/Finalised Version/"+dtype+\
                 "/Latest Features/"+ltype+"_Jaccard_Similarity.csv",sep=',',index=None,header=None)

In [0]:
def get_rouge_score(df,ltype,dtype):
  r1_recall = []
  r1_precision = []
  for i in range(df.shape[0]):
    if(i%10000 == 0):
      print(i)
    a = set(df['query'][i].split())
    b = set(df['response'][i].split())
    c = a.intersection(b)
    r1_recall.append(float (len(c)/len(a)))
    r1_precision.append(float (len(c)/len(b)))
  rouge_pd = pd.concat([pd.Series(r1_recall),pd.Series(r1_precision)],axis=1)
  rouge_pd.columns = ['Rouge R','Rouge P']
  print(rouge_pd.shape)
  rouge_pd.to_csv("./gdrive/My Drive/Microsoft AI/Finalised Version/"+dtype+\
                  "/Latest Features/"+ltype+"_Rouge_Score.csv",sep=',',index=None)

In [0]:
def TFIDFMatrix(docs):
  terms = []
  for i in range(len(docs)):
    words = docs[i].split()
    for w in words:
      if w not in terms:
        terms.append(w)
  
  tf = np.ndarray((len(terms),len(docs)))
  
  for i in range(len(terms)):
    for j in range(len(docs)):
      tf[i][j] = docs[j].split().count(terms[i])
      
  idf = np.array([np.log(1 + (float(len(docs))/np.count_nonzero(tf[i]))) for i in range(len(terms))])
  tf = 0.5 + 0.5*tf/tf.max(axis=0)  
  tfidf = np.array([tf[i]*idf[i] for i in range(len(terms))]).T
  return tfidf, terms

def get_tfidf_score(df,ltype,dtype):
  scores = []
  for i in range(0,df.shape[0],10):
    
    if(i%10000 == 0):
      print(i)
    
    docs = df['response'][i:i+10].tolist()
    mat, terms = TFIDFMatrix(docs)
    mat = pd.DataFrame(mat, columns=terms)
    
    words = df['query'][i].split()
    tf = {}
    for w in words:
      if w not in tf.keys():
        tf[w] = 1
      else:
        tf[w] += 1
    mx = max(tf.values())
    new_row = np.zeros(mat.shape[1])
    for k in tf.keys():
      tf[k] = 0.5+0.5*tf[k]/mx
      try:
        new_row[mat.columns.tolist().index(k)] = tf[k]
      except:
        pass
    mat = pd.DataFrame(np.vstack((new_row,mat.values)),columns=terms)
    scores.extend(cosine_similarity(mat.values)[0,1:])
  scores = pd.Series(scores).rename("Tfidf")
  print(scores.shape)
  scores.to_csv("./gdrive/My Drive/Microsoft AI/Finalised Version/"+dtype+\
                "/Latest Features/"+ltype+"_Tfidf_Score.csv",sep=',',index=None,header=None)

In [0]:
def get_bleu_score(df,ltype,dtype):
  bleu_score = []
  for i in range(df.shape[0]):
    if(i%10000 == 0):
      print(i)
    bleu_score.append([sentence_bleu([df['response'][i].split()],df['query'][i].split(),weights=(1,0)), sentence_bleu([df['response'][i].split()],df['query'][i].split(),weights=(1,0))])
  bleu_score = pd.DataFrame(bleu_score)
  bleu_score.columns = ['BLEU 1','BLEU 2']
  print(bleu_score.shape)
  bleu_score.to_csv("./gdrive/My Drive/Microsoft AI/Finalised Version/"+dtype+\
                    "/Latest Features/"+ltype+"_BLEU_Score.csv",sep=',',index=None)

In [0]:
def get_bm25_score(ltype,dtype):
  bm_data = pd.read_csv("./gdrive/My Drive/Microsoft AI/Finalised Version/"+dtype+"/Latest Features/"+ltype+"_BM25.tsv",sep='\t',header=None)
  bm = []
  for i in range(len(bm_data)):
    bm.extend(bm_data.iloc[i,1:].tolist())
  bm = pd.Series(bm).rename("BM25")
  print(bm.shape)
  bm.to_csv("./gdrive/My Drive/Microsoft AI/Finalised Version/"+dtype+\
            "/Latest Features/"+ltype+"_BM25.csv",sep=',',index=None,header=None)

In [0]:
def get_dssm_score(ltype,dtype):
  with open("./gdrive/My Drive/Microsoft AI/Finalised Version/"+dtype+"/Latest Features/DSSM/"+ltype+"_dssm.txt") as f:
    dvalues = f.readlines()
  with open("./gdrive/My Drive/Microsoft AI/Finalised Version/"+dtype+"/Latest Features/DSSM/"+ltype+"_cdssm.txt") as f:
    cvalues = f.readlines()
  dvalues = [float(val.split("\n")[0]) for val in dvalues]
  cvalues = [float(val.split("\n")[0]) for val in cvalues]
  if(ltype=='Train'):
    dssm = pd.concat([pd.Series(dvalues[1:]),pd.Series(cvalues[1:])],axis=1)
  else:
    dssm = pd.concat([pd.Series(dvalues),pd.Series(cvalues)],axis=1)
  dssm.columns = ['DSSM','SDSSM']
  print(dssm.shape)
  dssm.to_csv("./gdrive/My Drive/Microsoft AI/Finalised Version/"+dtype+\
              "/Latest Features/"+ltype+"_DSSM.csv",sep=',',index=None)

# Data Loading

In [0]:
data = pd.read_csv("gdrive/My Drive/Microsoft AI/Finalised Version/Hashed/Latest Data/clean_data_latest_#.csv",sep=',')
data.columns = ['queryID','query','response','label','labelID']

In [0]:
data2 = pd.read_csv("./gdrive/My Drive/Microsoft AI/Finalised Version/Hashed/Latest Data/clean_test_data_#.csv",sep=',')
data2.columns = ['queryID','query','response','labelID']

## Train Data Features Extraction

In [0]:
get_cosine_sim(data,'Train','Hashed')

In [0]:
get_jaccard_sim(data,'Train','Hashed')

In [0]:
get_rouge_score(data,'Train','Hashed')

In [0]:
get_tfidf_score(data,'Train','Hashed')

In [0]:
get_bleu_score(data,'Train','Hashed')

In [0]:
get_bm25_score("Train","Hashed")

In [0]:
get_dssm_score('Train','Hashed')

In [0]:
get_basic_features(data,'Train','Hashed')

## Test Data Features Extraction

In [0]:
get_cosine_sim(data2,'Test','Hashed')

In [0]:
get_jaccard_sim(data2,'Test','Hashed')

In [0]:
get_rouge_score(data2,'Test','Hashed')

In [0]:
get_tfidf_score(data2,'Test','Hashed')

In [0]:
get_bleu_score(data2,'Test','Hashed')

In [0]:
get_bm25_score("Test","Hashed")

In [0]:
get_dssm_score('Test','Hashed')

In [0]:
get_basic_features(data2,"Test","Hashed")

## Merge Features

In [0]:
def merge_features(data,ltype, dtype):
  path = os.path.join("gdrive/My Drive/Microsoft AI/Finalised Version",dtype,"Latest Features",ltype+"_")
  
  s1 = pd.read_csv(path+"QueryID.csv",sep=',',header=None).reset_index(drop=True)
  s2 = pd.read_csv(path+"Cosine_Similarity.csv",sep=',',header=None).reset_index(drop=True)
  s3 = pd.read_csv(path+"Jaccard_Similarity.csv",sep=',',header=None).reset_index(drop=True)
  s4 = pd.read_csv(path+"BM25.csv",sep=',',header=None).reset_index(drop=True)
  s5 = pd.read_csv(path+"Tfidf_Score.csv",sep=',',header=None).reset_index(drop=True)
  s6 = pd.read_csv(path+"Rouge_Score.csv",sep=',').reset_index(drop=True)
  s7 = pd.read_csv(path+"BLEU_Score.csv",sep=',').reset_index(drop=True)
  s8 = pd.read_csv(path+"DSSM.csv",sep=',').reset_index(drop=True)
  if(ltype == 'Train'):
    s9 = data['label'].reset_index(drop=True)
    s10 = pd.read_csv(path+"Basic_features.csv",sep=',',low_memory=False).drop(columns=['queryID','query','response','label','labelID']).reset_index(drop=True)
    all_feat = pd.concat([s1,s2,s3,s4,s5,s6,s7,s8,s9,s10],axis=1)
    all_feat.columns =  ['qid','Cosine','Jaccard','BM25','Tfidf','Rouge P','Rouge R','BLEU 1','BLEU 2','DSSM','CDSSM','label','q_word_count','q_char_count','q_avg_word','r_word_count','r_char_count','r_avg_word']
    all_feat.to_csv("gdrive/My Drive/Microsoft AI/Finalised Version/All_"+ltype+"Features.csv",sep=',',index=None)
  else:
    s9 = pd.read_csv(path+"Basic_features.csv",sep=',',low_memory=False).drop(columns=['queryID','query','response','labelID']).reset_index(drop=True)
    all_feat = pd.concat([s1,s2,s3,s4,s5,s6,s7,s8,s9],axis=1)
    all_feat.columns =  ['qid','Cosine','Jaccard','BM25','Tfidf','Rouge P','Rouge R','BLEU 1','BLEU 2','DSSM','CDSSM','q_word_count','q_char_count','q_avg_word','r_word_count','r_char_count','r_avg_word']
    all_feat.to_csv("gdrive/My Drive/Microsoft AI/Finalised Version/All_"+ltype+"Features.csv",sep=',',index=None)

In [0]:
merge_features(data,'Train','Hashed')

In [0]:
merge_features(data2,"Test","Hashed")

# Model and Hyper parameters

In [0]:
features = ['Cosine','Jaccard','BM25','Tfidf','Rouge P','Rouge R','BLEU 1','BLEU 2','DSSM','CDSSM',
            'q_word_count','q_char_count','q_avg_word','r_word_count','r_char_count','r_avg_word']

embed = pd.read_csv("gdrive/My Drive/Microsoft AI/Junk/Just For now/Train_Embeddings.csv")
data = pd.read_csv("gdrive/My Drive/Microsoft AI/Finalised Version/All_TrainFeatures.csv",usecols=features).iloc[:embed.shape[0]]
labelID = pd.read_csv("gdrive/My Drive/Microsoft AI/Finalised Version/Hashed/Latest Features/Train_labelID.csv",sep=',',header=None).iloc[:embed.shape[0]]
queryID = pd.read_csv("gdrive/My Drive/Microsoft AI/Finalised Version/Hashed/Latest Features/Train_QueryID.csv",sep=',',header=None).iloc[:embed.shape[0]]
combined_data = pd.concat([data,embed],axis=1)

split = int(10*((embed.shape[0]*0.7)//10))
print(split)

train_data, test_data = combined_data.iloc[:split].reset_index( drop = True), combined_data.iloc[split:].reset_index( drop = True)
train_labelID, test_labelID = labelID.iloc[:split].reset_index( drop = True), labelID.iloc[split:].reset_index( drop = True)
train_queryID, test_queryID = queryID.iloc[:split].reset_index( drop = True), queryID.iloc[split:].reset_index( drop = True)

tr_ind = train_data[train_data['labels']==1].index
fl_ind = train_data[train_data['labels']==0].index

tr_data, fl_data = train_data.iloc[tr_ind].reset_index( drop = True), train_data.iloc[fl_ind].reset_index( drop = True)

tr_ind = random.sample(range(0,tr_data.shape[0]),tr_data.shape[0])
fl_ind = random.sample(range(0,fl_data.shape[0]),tr_data.shape[0]*4)

train_data = pd.concat([tr_data.iloc[tr_ind],fl_data.iloc[fl_ind]],axis=0).sample(frac=1).reset_index( drop = True)

train_f_data, test_f_data = train_data.values[:,:16], test_data.values[:,:16]
train_q_embed, test_q_embed = train_data.values[:,46:76], test_data.values[:,46:76]
train_r_embed, test_r_embed = train_data.values[:,16:46], test_data.values[:,16:46]
train_labels, test_labels = train_data.values[:,-1], test_data.values[:,-1]

print(train_q_embed.shape)
print(train_r_embed.shape)
print(train_f_data.shape)
print(test_q_embed.shape)
print(test_r_embed.shape)
print(test_f_data.shape)
print(train_labels.shape)
print(test_labels.shape)
print(test_labelID.shape)
print(test_queryID.shape)

In [0]:
features = ['Cosine','Jaccard','BM25','Tfidf','Rouge P','Rouge R','BLEU 1','BLEU 2','DSSM','CDSSM',
            'q_word_count','q_char_count','q_avg_word','r_word_count','r_char_count','r_avg_word']

data = pd.read_csv("gdrive/My Drive/Microsoft AI/Finalised Version/All_TrainFeatures.csv")
split = int(10*((data.shape[0]*0.7)//10))
print(split)

train_data, test_data = data.iloc[:split]

tr_data = data[data['label'] == 1].reset_index(drop=True)
fl_data = data[data['label'] == 0].reset_index(drop=True)
tr_ind = random.sample(range(0,tr_data.shape[0]),tr_data.shape[0])
fl_ind = random.sample(range(0,fl_data.shape[0]),tr_data.shape[0]*3)
df = pd.concat([tr_data.iloc[tr_ind],fl_data.iloc[fl_ind]],axis=0).sample(frac=1).reset_index( drop = True)

y = df['label']
x = df[features]
print(len(features))

# x = (x - x.min()) / (x.max() - x.min())
x = (x - df.mean()) / df.std()
x_train, x_test = x.values[:split,:], x.values[split:,:]
y_train, y_test = y.values[:split], y.values[split:]
x_train.shape, y_train.shape, x_test.shape, y_test.shape

## Hyper Parameter Tuning

In [0]:
hyperparams = {
    "trees":{
        "criterions" : ["gini","entropy"],
        "max_depth" : [3,4,5,6,7,8,9,10],
        "max_features": ["log2","sqrt","auto"], 
        "n_estimators": [20,40,60,80,100]}, 
    "gbc":{
        "loss" : ["deviance","exponential"],
        "learning_rate" : [0.1,0.5,0.8,1.0],
        "n_estimators" : [20,40,60,80,100],
        "max_features" : ["log2","sqrt","auto"],
        "max_depth" : [3,4,5,6,7,8,9,10]},
    "nn":{
        "learning_rate" : [0.1,0.01,0.05,0.001,0.005],
        "optim" : ["Adam","Adamax","Adagrad","Nadam"], 
        "activation": ["tanh","sigmoid","relu","linear"]}
}

results = {}
results['RFC'] = {}

for cr in hyperparams["trees"]["criterions"]:
    results['RFC'][str(cr)] = {}
    for dep in hyperparams["trees"]["max_depth"]:
        results['RFC'][str(cr)][str(dep)] = {}
        for feat in hyperparams["trees"]["max_features"]:
            results['RFC'][str(cr)][str(dep)][str(feat)] = {}
            for est in hyperparams["trees"]["n_estimators"]:
                results['RFC'][str(cr)][str(dep)][str(feat)][str(est)] = {}
                print("Hyperparameters used")
                print("Criterion : ",cr)
                print("Maximum Depth : ",dep)
                print("Maximum Features : ",feat)
                print("Number of Estimators : ",est)
                model = RandomForestClassifier(n_estimators=est,
                                              criterion=cr,
                                              max_features=feat,
                                              max_depth=dep,
                                              verbose=3,
                                              class_weight="balanced")
                
                model.fit(x_train,y_train)
                y_pred = model.predict(x_test)
                imp = model.feature_importances_
                score = model.score(x_test,y_test)
                print(score)
                results['RFC'][str(cr)][str(dep)][str(feat)][str(est)]["score"] = score
                
                f = pd.DataFrame(np.column_stack((features,imp)))
                f.columns = ['feature','importance']
                print(f.sort_values(by=['importance'],ascending=False))
                results['RFC'][str(cr)][str(dep)][str(feat)][str(est)]["f_imp"] = f

results['ETC'] = {}

for cr in hyperparams["trees"]["criterions"]:
    results['ETC'][str(cr)] = {}
    for dep in hyperparams["trees"]["max_depth"]:
        results['ETC'][str(cr)][str(dep)] = {}
        for feat in hyperparams["trees"]["max_features"]:
            results['ETC'][str(cr)][str(dep)][str(feat)] = {}
            for est in hyperparams["trees"]["n_estimators"]:
                results['ETC'][str(cr)][str(dep)][str(feat)][str(est)] = {}
                print("Hyperparameters used")
                print("Criterion : ",cr)
                print("Maximum Depth : ",dep)
                print("Maximum Features : ",feat)
                print("Number of Estimators : ",est)
                model = ExtraTreesClassifier(n_estimators=est,
                                              criterion=cr,
                                              max_features=feat,
                                              max_depth=dep,
                                              verbose=3,
                                              warm_start=True,
                                              class_weight="balanced")
                
                model.fit(x_train,y_train)
                y_pred = model.predict(x_test)
                imp = model.feature_importances_
                score = model.score(x_test,y_test)
                print(score)
                results['ETC'][str(cr)][str(dep)][str(feat)][str(est)]["score"] = score
                
                f = pd.DataFrame(np.column_stack((features,imp)))
                f.columns = ['feature','importance']
                print(f.sort_values(by=['importance'],ascending=False))
                results['ETC'][str(cr)][str(dep)][str(feat)][str(est)]["f_imp"] = f

results['GBC'] = {}

for ls in hyperparams["gbc"]["loss"]:
    results['GBC'][str(ls)] = {}
    for lr in hyperparams["gbc"]["learning_rate"]:
        results['GBC'][str(ls)][str(lr)] = {}
        for est in hyperparams["gbc"]["n_estimators"]:
            results['GBC'][str(ls)][str(lr)][str(est)] = {}
            for feat in hyperparams["gbc"]["max_features"]:
                results['GBC'][str(ls)][str(lr)][str(est)][str(feat)] = {}
                for dep in hyperparams["gbc"]["max_depth"]:
                    results['GBC'][str(ls)][str(lr)][str(est)][str(feat)][str(dep)] = {}
                    print("Hyperparameters used")
                    print("Loss : ",ls)
                    print("Learning rate : ",lr)
                    print("Number of Estimators : ",est)
                    print("Maximum Features : ",feat)
                    print("Maximum Depth : ",dep)
                    model = GradientBoostingClassifier(loss=ls,
                                                       learning_rate=lr,
                                                       n_estimators=est,
                                                       max_features=feat,
                                                       max_depth=dep,
                                                       verbose=3)

                    model.fit(x_train,y_train)
                    y_pred = model.predict(x_test)
                    imp = model.feature_importances_
                    score = model.score(x_test,y_test)
                    print(score)
                    results['GBC'][str(ls)][str(lr)][str(est)][str(feat)][str(dep)]["score"] = score

                    f = pd.DataFrame(np.column_stack((features,imp)))
                    f.columns = ['feature','importance']
                    print(f.sort_values(by=['importance'],ascending=False))
                    results['GBC'][str(ls)][str(lr)][str(est)][str(feat)][str(dep)]["f_imp"] = f

# tree models tabulation
columns = ["model type","criterion","learning rate","max features","max depth","# of estimators","score"]
l = []
for mdl in results.keys():
    if(mdl != "GBC"):
        for crt in results[mdl].keys():
            for mfeat in results[mdl][crt].keys():
                for mdep in results[mdl][crt][mfeat].keys():
                    for nest in results[mdl][crt][mfeat][mdep].keys():
                        score = results[mdl][crt][mfeat][mdep][nest]["score"]
                        l.append([mdl,crt,None,mfeat,mdep,nest,score])
    else:
        for crt in results[mdl].keys():
            for lr in results[mdl][crt].keys():
                for nest in results[mdl][crt][lr].keys():
                    for mfeat in results[mdl][crt][lr][nest].keys():
                        for mdep in results[mdl][crt][lr][nest][mfeat].keys():
                            score = results[mdl][crt][lr][nest][mfeat][mdep]["score"]
                            l.append([mdl,crt,lr,mfeat,mdep,nest,score])

res = pd.DataFrame(l,columns=columns)
res = res.sort_values(by=['score'],ascending=False).reset_index(drop=True)

res.to_csv("./Tree_Hyper.csv",sep=',',index=None)

In [0]:
results['nn'] = {}
for optim in hyperparams['nn']['optim']:
    results['nn'][str(optim)] = {}
    for lr in hyperparams['nn']['learning_rate']:
        results['nn'][str(optim)][str(lr)] = {}
        for act in hyperparams['nn']['activation']:
            print("Hyperparameter used")
            print("Optimizer : ",optim)
            print("Learning rate : ",lr)
            print("Activation : ",act)
            results['nn'][str(optim)][str(lr)][str(act)] = {}
            inp = L.Input(shape=(len(features),))
            x = L.Dense(512, activation=act)(inp)
            x = L.Dense(256, activation=act)(x)
            x = L.Dense(128, activation=act)(x)
            x = L.Dense(128, activation=act)(x)
            x = L.Dense(64, activation=act)(x)
            out = L.Dense(1, activation="sigmoid")(x)
            
            model = M.Model(inputs=[inp],outputs=[out])
            model.summary()
            if optim == "Adam":
                o = Adam(lr)
            elif optim == "Adagrad":
                o = Adagrad(lr)
            elif optim == "Adamax":
                o = Adamax(lr)
            elif optim == "Nadam":
                o = Nadam(lr)
            model.compile(o,loss='binary_crossentropy',metrics=['accuracy'])
            
            model.fit(x_train,y_train,epochs=10)
            score = model.evaluate(x_test,y_test)
            y_pred = model.predict(x_test)
            
            results['nn'][str(optim)][str(lr)][str(act)]["score"] = score

# tree models tabulation
columns = ["optimizer","learning rate","activation","score"]
l = []
for mdl in results.keys():
    if(mdl == "nn"):
        for optim in results[mdl].keys():
            for lr in results[mdl][optim].keys():
                for act in results[mdl][optim][lr].keys():
                    score = results[mdl][optim][lr][act]["score"]
                    l.append([optim,lr,act,score])

res = pd.DataFrame(l,columns=columns)
res["loss"] = [s[0] for s in res["score"]]
res["score"] = [s[1] for s in res["score"]]

res = res.sort_values(by=['score'],ascending=False).reset_index(drop=True)
res.to_csv("NN_Hyper.csv",sep=',',index=None)

## Model Training

In [0]:
#model type,criterion,learning rate,max features,max depth,# of estimators,score
#GBC,exponential,0.1,log2,7,100,0.900060606060606
#RFC,gini,,10,auto,80,0.6947878787878788
ls = "exponential"
lr = 0.1
est = 100
feat = "log2"
dep = 7

# model1 = RandomForestClassifier(n_estimators=est,criterion=cr,max_features=feat,max_depth=dep,verbose=3,class_weight="balanced_subsample")
model1 = GradientBoostingClassifier(loss=ls, learning_rate=lr,n_estimators=est,max_features=feat,max_depth=dep,verbose=3)
# model1 = ExtraTreesClassifier(n_estimators=100, criterion='gini', verbose=0,n_jobs=-1, max_features='auto', min_samples_split=3, min_samples_leaf=20)

model1.fit(x_train,y_train)

y_pred = model1.predict(x_test)
imp = model1.feature_importances_
score = model1.score(x_test,y_test)
print(score)

pd.DataFrame(np.column_stack((features,imp)))

In [0]:
def model_construct1(q_inp_size, r_inp_size, feature_size, mode='train'):
  inp1 = tf.keras.layers.Input(shape=q_inp_size)
  inp2 = tf.keras.layers.Input(shape=r_inp_size)
  inp3 = tf.keras.layers.Input(shape=feature_size)

  x3 = tf.keras.layers.Dense(1024,activation='relu')(inp3)
  x3 = tf.keras.layers.Dense(512,activation='relu')(x3)
  x3 = tf.keras.layers.Dense(256,activation='relu')(x3)
  x3 = tf.keras.layers.Dense(128,activation='relu')(x3)
  
  f1 = tf.keras.layers.Add()([inp1,inp2])
  f2 = tf.keras.layers.Multiply()([inp1,inp2])
  f3 = tf.keras.layers.Subtract()([inp1,inp2])
  
  f1 = tf.keras.layers.Lambda(lambda x: tf.keras.backend.expand_dims(x,axis=-1))(f1)
  f2 = tf.keras.layers.Lambda(lambda x: tf.keras.backend.expand_dims(x,axis=-1))(f2)
  f3 = tf.keras.layers.Lambda(lambda x: tf.keras.backend.expand_dims(x,axis=-1))(f3)
  inps1 = tf.keras.layers.Lambda(lambda x: tf.keras.backend.expand_dims(x,axis=-1))(inp1)
  inps2 = tf.keras.layers.Lambda(lambda x: tf.keras.backend.expand_dims(x,axis=-1))(inp2)

  comb = tf.keras.layers.Concatenate(axis=-1)([inps1,inps2,f1,f2,f3])

  x1 = tf.keras.layers.Conv1D(64,5,activation='relu')(comb)
  x1 = tf.keras.layers.Conv1D(32,3,activation='relu')(x1)
  
  x2 = tf.keras.layers.GRU(32,activation='relu',return_sequences=True)(comb)
  x2 = tf.keras.layers.GRU(32,activation='relu',return_sequences=True)(x2)
  
  comb = tf.keras.layers.Concatenate(axis=1)([x1,x2])
  
  x1 = tf.keras.layers.Flatten()(comb)
  x1 = tf.keras.layers.Dense(1024,activation='relu')(x1)
  x1 = tf.keras.layers.Dense(512,activation='relu')(x1)
  x1 = tf.keras.layers.Dense(256,activation='relu')(x1)
  x1 = tf.keras.layers.Dense(128,activation='relu')(x1)
  
  x = tf.keras.layers.Concatenate(axis=-1)([x1,x3])
  
  x = tf.keras.layers.Dense(512,activation='relu')(x)
  x = tf.keras.layers.Dense(256,activation='relu')(x)
  out = tf.keras.layers.Dense(1,activation='sigmoid')(x)

  model = tf.keras.models.Model(inputs=[inp1,inp2,inp3],outputs=[out],name="Embedding Model with Features")
  if mode == 'train':
    model.summary()
  return model

In [0]:
def model_construct2(features):
  inp = tf.keras.layers.Input(shape=(len(features),))
  x = tf.keras.layers.Dense(256, activation='sigmoid')(inp)
  x = tf.keras.layers.Dense(512, activation='sigmoid')(x)
  x = tf.keras.layers.Dense(1024, activation='sigmoid')(x)
  out = tf.keras.layers.Dense(1, activation='sigmoid')(x)

  model = tf.keras.models.Model(inputs=[inp],outputs=[out])
  model.summary()

  return model

In [0]:
keras_model = model_construct1((train_q_embed.shape[1],),(train_r_embed.shape[1],),(len(features),))
tf.keras.backend.clear_session()
tpu_model = tf.contrib.tpu.keras_to_tpu_model(keras_model, 
                                              strategy=tf.contrib.tpu.TPUDistributionStrategy(
                                                  tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

tpu_model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])
tpu_model.fit([train_q_embed,train_r_embed,train_f_data],train_labels,epochs=10,batch_size=128,
              validation_data=([test_q_embed,test_r_embed,test_f_data],test_labels))

tpu_model.save_weights('./features_embeddings_bi-model.h5', overwrite=True)
! cp -fR "./features_embeddings_bi-model.h5" "./gdrive/My Drive/Microsoft AI/Junk/"

# Evaluation Code

In [0]:
def find_n_save_prob(qids,mtype,fname,x_test_actual=None,model=None,eval_gen=None):
  if(mtype=='tree'):
    assert model!=None,"Model is None"
    probs = model.predict_proba(x_test_actual)[:,1]
  elif(mtype=='simple_nn'):
    model = model_construct2(features)
    model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])
    model.load_weights("./gdrive/My Drive/Microsoft AI/Junk/features_embeddings_bi-model.h5")
    probs = model.predict(x_test_actual).ravel()
  elif(mtype=='em_nn_whole'):
    model = model_construct1((30,),(30,),(16,),'eval')
    model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])
    model.load_weights("./gdrive/My Drive/Microsoft AI/Junk/features_embeddings_bi-model.h5")
    probs = model.predict(x_test_actual).ravel()
  else:
    model = model_construct1((30,),(30,))
    model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])
    model.load_weights("./gdrive/My Drive/Microsoft AI/Junk/features_embeddings_bi-model.h5")
    probs = model.predict_generator(eval_gen).ravel()
  
  p = []
  if(mtype=='em_nn_whole'):
    t = x_test_actual[0]
  else:
    t = x_test_actual
  for i in range(0,t.shape[0],10):
      l = []
      v = probs[i:i+10].T
      l.append(qids[i])
      l.extend(v)
      p.append(l)

  result = pd.DataFrame(p)
  
  try:
    os.mkdir(os.path.join("gdrive/My Drive/Microsoft AI/Results",mtype))
  except:
    pass
  try:
    os.mkdir(os.path.join("gdrive/My Drive/Microsoft AI/Results",mtype,fname))
  except:
    pass
  save_name = os.path.join("gdrive/My Drive/Microsoft AI/Results",mtype,fname,"answer.tsv")
  result.to_csv(save_name,sep='\t',header=None,index=False)

In [0]:
data2 = pd.read_csv("gdrive/My Drive/Microsoft AI/Finalised Version/All_TestFeatures.csv",sep=',')
qids = pd.read_csv("./gdrive/My Drive/Microsoft AI/Finalised Version/Hashed/Latest Features/Test_QueryID.csv").values.ravel()
x_test_actual = data2[features]
find_n_save_prob(qids,x_test_actual,"nn","26-12_NN_with_all_feat_norm")

In [0]:
qids = pd.read_csv("./gdrive/My Drive/Microsoft AI/Finalised Version/Hashed/Latest Features/Test_QueryID.csv").values.ravel()
eval_q_embed = pd.read_csv("gdrive/My Drive/Microsoft AI/Junk/Just For now/Test_Query_Embeddings.csv")
eval_r_embed = pd.read_csv("gdrive/My Drive/Microsoft AI/Junk/Just For now/Test_Response_Embeddings.csv")
eval_f_data = pd.read_csv("gdrive/My Drive/Microsoft AI/Finalised Version/All_TestFeatures.csv",sep=',',usecols=features)
find_n_save_prob(qids,"em_nn_whole","Features_Embedding_BiDir_Based",x_test_actual=[eval_q_embed,eval_r_embed,eval_f_data])

In [0]:
model = model_construct1((30,),(30,),(16,),'eval')
model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])
model.load_weights("./gdrive/My Drive/Microsoft AI/Junk/features_embeddings_bi-model.h5")
probs = model.predict([test_q_embed,test_r_embed,test_f_data]).ravel()

In [0]:
p = []
for i in range(0,probs.shape[0],10):
  l = []
  v = probs[i:i+10].T
  l.append(test_queryID.values[i][0])
  l.extend(v)
  p.append(l)

p = np.array(p)
y_pred = pd.DataFrame(p[:,1:],index=p[:,0].astype(int))

In [0]:
tmp=pd.DataFrame(test_labelID.values.ravel(),index=test_queryID.astype(int).values.ravel()).reset_index(drop=True)
h = []
for i in range(0,tmp.shape[0],10):
  h.append(tmp.values[i])
y_true = pd.DataFrame(h)

### MRR Score

In [0]:
y_pred.columns = list(range(10))
y_true.columns = ['labelID']

In [0]:
def MRR(sub,ref):
  scores = []
  for q_id in ref.index:
    position = ref.iloc[q_id].values.ravel()
    #print(position)
    rank = np.argsort(sub.iloc[q_id])[::-1].tolist().index(position)
    rank +=1
    scores.append(1.0/rank)

  score = np.mean(scores)
  print(score)

In [0]:
MRR(y_pred,y_true)